In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leinad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import numpy
import tflearn
import tensorflow as tf
import random
import json
import pickle
from tensorflow import keras

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
with open("intents.json") as file:
	data = json.load(file)

In [4]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
	for pattern in intent["patterns"]:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])

	if intent["tag"] not in labels:
		labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []

	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

In [13]:
labels

['Abdonominal Pain',
 'Abrasions',
 'Broken Toe',
 'Bruises',
 'CPR',
 'Chemical Burn',
 'Choking',
 'Cold',
 'Cough',
 'Cuts',
 'Diarrhea',
 'Drowning',
 'Eye Injury',
 'Fainting',
 'Fever',
 'Fracture',
 'Frost bite',
 'Gastrointestinal problems',
 'Head Injury',
 'Headache',
 'Heat Exhaustion',
 'Heat Stroke',
 'Insect Bites',
 'Nasal Congestion',
 'Normal Bleeding',
 'Poison',
 'Pulled Muscle',
 'Rash',
 'Rectal bleeding',
 'Skin problems',
 'Sore Throat',
 'Splinter',
 'Sprains',
 'Strains',
 'Sun Burn',
 'Teeth',
 'Testicle Pain',
 'Vertigo',
 'Wound',
 'animal bite',
 'nose bleed',
 'seizure',
 'snake bite',
 'stings']

In [5]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
try:
	model.load("model.tflearn")
except:
	model = tflearn.DNN(net)
	model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
	model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from c:\Users\Leinad\Documents\Tech_Villain\Leinadpython\firstaid\model.tflearn


In [7]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [8]:
def chat():
	print("Start Talking with the bot(type quit to stop!")
	while True:
		inp = input("You: ")
		if inp.lower() == "quit":
			break

		results = model.predict([bag_of_words(inp,words)])[0]
		results_index = numpy.argmax(results)
		tag = labels[results_index]

		if results[results_index] > 0.5:
			for tg in data["intents"]:
				if tg['tag'] == tag:
					responses = tg['responses']	
			print(random.choice(responses))
			print("\n")
		else:
			print("I didnt get that, try again")


In [23]:
chat()

Start Talking with the bot(type quit to stop!
To treat a fever at home:
1. Drink plenty of fluids to stay hydrated.
2. Dress in lightweight clothing.
3. Use a light blanket if you feel chilled, until the chills end.
4. Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others).
5. Get medical help if the fever lasts more than five days in a row.




In [11]:
from tensorflow.keras.models import load_model

model.save('C:\\Users\\Leinad\\Documents\\Tech_Villain\\Leinadpython\\firstaid\\model.hdf5')

INFO:tensorflow:C:\Users\Leinad\Documents\Tech_Villain\Leinadpython\firstaid\model.hdf5 is not in all_model_checkpoint_paths. Manually adding it.


In [25]:
# Assuming you have already trained the DNN model and loaded it as 'model'.
# Also, you have prepared the test dataset as 'test_data' and 'test_labels'.

# Convert labels to one-hot encoded vectors
label_dict = {label: i for i, label in enumerate(labels)}
test_labels_encoded = [[1 if label_dict[label] == i else 0 for i in range(len(labels))] for label in test_labels]

# Preprocess the test data similar to the training data
test_input = []
for sentence in test_data:
    bag = bag_of_words(sentence, words)
    test_input.append(bag)

# Convert test_input to a numpy array
test_input = numpy.array(test_input)

# Make predictions using the model
predictions = model.predict(test_input)

# Calculate the accuracy
correct_predictions = 0
total_predictions = len(test_labels)
for i in range(total_predictions):
    predicted_label_index = numpy.argmax(predictions[i])
    true_label_index = numpy.argmax(test_labels_encoded[i])
    if predicted_label_index == true_label_index:
        correct_predictions += 1

accuracy = correct_predictions / total_predictions
print("Accuracy: {:.2f}%".format(accuracy * 100))

# evaluation_result = model.evaluate(test_input, test_labels_encoded)
# loss = evaluation_result[0]  # Extract the first element (loss value) from the evaluation result list
# print("Loss: {:.4f}".format(loss))


Accuracy: 100.00%


In [10]:
from sklearn.model_selection import train_test_split
train_data, eval_data, train_labels, eval_labels = train_test_split(training, output, test_size=0.2, random_state=42)

In [12]:
from sklearn.metrics import accuracy_score
predictions = model.predict(eval_data)

# Convert the predictions to class labels (index with the highest probability)
predicted_labels = [labels[prediction.argmax()] for prediction in predictions]

# Calculate accuracy as the evaluation metric
accuracy = accuracy_score([labels[label.argmax()] for label in eval_labels], predicted_labels)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [14]:
import numpy as np
def calculate_tp_fp_fn(y_true, y_pred):
    tp = np.sum(np.logical_and(y_true == 1, y_pred == 1))
    fp = np.sum(np.logical_and(y_true == 0, y_pred == 1))
    fn = np.sum(np.logical_and(y_true == 1, y_pred == 0))
    return tp, fp, fn

# Evaluate precision and recall
def evaluate_precision_recall(y_true, y_pred):
    tp, fp, fn = calculate_tp_fp_fn(y_true, y_pred)
    precision = tp / (tp + fp + 1e-8)  # Add small epsilon to avoid division by zero
    recall = tp / (tp + fn + 1e-8)     # Add small epsilon to avoid division by zero
    return precision, recall

# Assuming you already have the predictions and ground truth labels
# predictions contains the predicted labels (converted to class indices)
# eval_labels contains the ground truth labels (converted to class indices)

# Convert predictions and eval_labels to binary format for multilabel evaluation
y_pred_binary = np.zeros_like(predictions)
y_pred_binary[np.arange(len(predictions)), predictions.argmax(axis=1)] = 1

eval_labels_binary = np.zeros_like(eval_labels)
eval_labels_binary[np.arange(len(eval_labels)), eval_labels.argmax(axis=1)] = 1

# Evaluate precision and recall
precision, recall = evaluate_precision_recall(eval_labels_binary, y_pred_binary)

print("Precision:", precision)
print("Recall:", recall)

Precision: 0.9999999997368421
Recall: 0.9999999997368421
